In [1]:
from final_model import WAE
import torch
import librosa as li

In [2]:
mel = torch.from_numpy(li.filters.mel(22050, 2048, n_mels=500))

In [3]:
torch.nn.Module.dump_patches = True
model = torch.load("model.pt", map_location="cpu")
model.eval()
print("Done!")

Done!


/home/antoine/lap/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'final_model.WAE' has changed. Tried to save a patch, but couldn't create a writable file WAE.patch. Make sure it doesn't exist and your working directory is writable.
  warnings.warn(msg, SourceChangeWarning)


In [4]:
model.forward = model.sample
model(torch.Tensor([1,2,1,2]).unsqueeze(0), torch.Tensor([.5]).unsqueeze(0))

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0461, 0.0351, 0.0328,  ..., 0.0326, 0.0337, 0.0470],
        [0.0457, 0.0338, 0.0316,  ..., 0.0305, 0.0311, 0.0448],
        ...,
        [0.0015, 0.0014, 0.0014,  ..., 0.0014, 0.0014, 0.0016],
        [0.0007, 0.0007, 0.0007,  ..., 0.0007, 0.0007, 0.0008],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<MmBackward>)

In [5]:
traced_model = torch.jit.trace(model, [torch.Tensor([1,2,1,2]).unsqueeze(0), torch.Tensor([.5]).unsqueeze(0)] )

/home/antoine/dev/serge/src/final_model.py:178: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mel = torch.from_numpy(li.filters.mel(22050, 2048, n_mels=500)).float()


In [6]:
traced_model.save("model_moins_bidon.pt")